In [ ]:
import numpy as np
import tensorflow as tf
import re
from collections import Counter
import json
from pprint import pprint
from tensorflow.contrib import learn
import re
import csv

# Tokenisation

In [ ]:
def tokenize_(s):
    pattern = r'''\d+|[`\-=~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?]|[A-Z][A-Z]+|http[s]?://[\w\./]+|[\w]+@[\w]+\.[\w]+|[a-z][a-z]+|[A-Za-z]\.[\w][\w\.]+|[\w]+|[-'a-z]+|[\S]+'''
    
    l = re.findall(pattern, s)
    return l

#Variables
sentence_train = []
score_train = []
sentence_test = []
score_test = []
sentence_valid = []
score_valid = []

# Reading Chat Dataset (from csv to list)

In [ ]:
filename = 'train.csv'

with open(filename, 'r') as f:
    next(f)
    reader = csv.reader(f, delimiter='$')
    
    for row in reader:
        score_train.append(int(row[0]))
        sentence_train.append(row[1])

filename = 'valid.csv'
with open(filename, 'r') as f:
    next(f)
    reader = csv.reader(f, delimiter='$')
    
    for row in reader:
        score_valid.append(int(row[0]))
        sentence_valid.append(row[1])

filename = 'test.csv'
with open(filename, 'r') as f:
    next(f)
    reader = csv.reader(f, delimiter='$')
    
    for row in reader:
        score_test.append(int(row[0]))
        sentence_test.append(row[1])

# Reading Glove (common crawl)

In [ ]:
filename = 'glove.840B.300d.txt'
def loadGloVe(filename):
    vocab = []
    embd = []
    file = open(filename,'r')
    for line in file.readlines():
        row = line.strip().split(' ')
        if (len(row[1:]) == 300):
            vocab.append(row[0])
            embd.append(row[1:])
    print('Loaded GloVe!')
    file.close()
    return vocab,embd

vocab,embd = loadGloVe(filename)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
vocab = set(vocab)
word_to_int = {word:i for i,word in enumerate(vocab,1)}


# Converting FULL Sentence to its Integer representation (not embedding)

In [ ]:
################## to make features of the train dataset #####################
sentence2ints = []
for each in sentence_train:
    each = tokenize_(each)
    this_sentence_int = []
    for word in each:
        if word in vocab:
            this_sentence_int.append(word_to_int[word])
    sentence2ints.append(this_sentence_int)

max_seq_len = 20
features_train = np.zeros((len(sentence2ints), max_seq_len), dtype=int)
for i, row in enumerate(sentence2ints):
    features_train[i, :len(row)] = np.array(row[:max_seq_len] )
#######################################################################


################## to make features of the test dataset #####################
sentence2ints = []
for each in sentence_test:
    each = tokenize_(each)
    this_sentence_int = []
    for word in each:
        if word in vocab:
            this_sentence_int.append(word_to_int[word])
    sentence2ints.append(this_sentence_int)

max_seq_len = 20
features_test = np.zeros((len(sentence2ints), max_seq_len), dtype=int)
for i, row in enumerate(sentence2ints):
    features_test[i, :len(row)] = np.array(row[:max_seq_len] )
#######################################################################


################## to make features of the valid dataset #####################
sentence2ints = []
for each in sentence_valid:
    each = tokenize_(each)
    this_sentence_int = []
    for word in each:
        if word in vocab:
            this_sentence_int.append(word_to_int[word])
    sentence2ints.append(this_sentence_int)

max_seq_len = 20
features_val = np.zeros((len(sentence2ints), max_seq_len), dtype=int)
for i, row in enumerate(sentence2ints):
    features_val[i, :len(row)] = np.array(row[:max_seq_len] )
#######################################################################


# Converting FULL Sentence to its Embedding

In [ ]:

#Build Look up table
W = tf.Variable(tf.constant(0.0, shape=[vocab_size, embedding_dim]),
                trainable=False, name="W")
embedding_placeholder = tf.placeholder(tf.float32, [vocab_size, embedding_dim])
embedding_init = W.assign(embedding_placeholder)

X = tf.placeholder(tf.int32, [None, None], name = 'inputs')
embed = tf.nn.embedding_lookup(W, X)

#Start Session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    _,em_train = sess.run([embedding_init, embed], feed_dict={embedding_placeholder:embedding, X:features_train})
    _,em_test = sess.run([embedding_init, embed], feed_dict={embedding_placeholder:embedding, X:features_test})
    _,em_val = sess.run([embedding_init, embed], feed_dict={embedding_placeholder:embedding, X:features_val})

# Saving Chat Dataset's as its Embedding

In [ ]:
import pickle

train_x = open('train_x.pkl', 'wb')
pickle.dump(em_train, train_x)
train_x.close()


val_x = open('val_x.pkl', 'wb')
pickle.dump(em_val, val_x)
val_x.close()


test_x = open('test_x.pkl', 'wb')
pickle.dump(em_test, test_x)
test_x.close()

train_y = open('train_y.pkl', 'wb')
pickle.dump(score_train, train_y)
train_y.close()


test_y = open('test_y.pkl', 'wb')
pickle.dump(score_test, test_y)
test_y.close()


val_y = open('val_y.pkl', 'wb')
pickle.dump(score_valid, val_y)
val_y.close()
